In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset
import torch.nn as nn
from transformers import BertModel
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertPreTrainedModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
from torch.cuda.amp import GradScaler, autocast
import gc
from transformers import DistilBertTokenizerFast, DistilBertModel, DistilBertConfig
import torch.nn as nn
import random
from sklearn.metrics import classification_report
import os
from transformers import DistilBertForSequenceClassification
print("Librerías cargadas exitosamente.")

Librerías cargadas exitosamente.


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.6.0+cu124
True


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Aplicaciones de data science/dataminiconda/train_data_limpioof.csv')
print("Dataset 'train_data_limpioof.csv' cargado exitosamente.")
print("\nPrimeras 5 filas del dataset:")

print(df.head())

print("\nInformación del dataset (columnas y tipos de datos):")
df.info()

print("\nVerificando valores nulos después de la carga:")
print(df.isnull().sum())

Dataset 'train_data_limpioof.csv' cargado exitosamente.

Primeras 5 filas del dataset:
       textID                                               text  \
0  cb774db0d1                  id have responded if i were going   
1  549e992a42         sooo sad i will miss you here in san diego   
2  088c60f138                             my boss is bullying me   
3  9642c003ef                      what interview leave me alone   
4  358bd9e861   sons of  why couldnt they put them on the rel...   

                       selected_text sentiment  
0  id have responded if i were going   neutral  
1                           sooo sad  negative  
2                        bullying me  negative  
3                     leave me alone  negative  
4                           sons of   negative  

Información del dataset (columnas y tipos de datos):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  

In [ ]:
# --- INICIO DE LA MODIFICACIÓN PARA BORRAR NaNs EN 'selected_text' ---
initial_rows = len(df)
df.dropna(subset=['selected_text'], inplace=True)
rows_after_dropna = len(df)
dropped_rows_count = initial_rows - rows_after_dropna

if dropped_rows_count > 0:
    print(f"\n¡Correcto! Se han eliminado {dropped_rows_count} filas con valores nulos en 'selected_text'.")
else:
    print("\nNo se encontraron valores nulos en 'selected_text' para eliminar.")


¡Correcto! Se han eliminado 95 filas con valores nulos en 'selected_text'.


In [ ]:
# --- FIN DE LA MODIFICACIÓN ---

print("\nVerificando valores nulos DESPUÉS DE LA LIMPIEZA ESPECÍFICA:")
print(df.isnull().sum()) # Volver a verificar para confirmar que selected_text es 0

print("\nPrimeras 5 filas del dataset después de la limpieza:")
print(df.head())
print("\nInformación del dataset (columnas y tipos de datos después de limpieza):")
df.info()


Verificando valores nulos DESPUÉS DE LA LIMPIEZA ESPECÍFICA:
textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

Primeras 5 filas del dataset después de la limpieza:
       textID                                               text  \
0  cb774db0d1                  id have responded if i were going   
1  549e992a42         sooo sad i will miss you here in san diego   
2  088c60f138                             my boss is bullying me   
3  9642c003ef                      what interview leave me alone   
4  358bd9e861   sons of  why couldnt they put them on the rel...   

                       selected_text sentiment  
0  id have responded if i were going   neutral  
1                           sooo sad  negative  
2                        bullying me  negative  
3                     leave me alone  negative  
4                           sons of   negative  

Información del dataset (columnas y tipos de datos después de limpieza):
<class 'pandas.cor

In [ ]:
# Una verificación final para asegurar que las columnas cruciales existen
if 'text' not in df.columns or 'selected_text' not in df.columns or 'sentiment' not in df.columns:
    raise ValueError("El dataset debe contener las columnas 'text', 'selected_text' y 'sentiment'.")


In [ ]:
# inicializar el tokenizador de Bert
MODEL_NAME = 'distilbert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

print(f"Tokenizador '{MODEL_NAME}' cargado exitosamente.")
print(f"Ejemplo de tokenización de una palabra: {tokenizer.tokenize('hola')}")
print(f"Ejemplo de tokenización de una frase: {tokenizer.tokenize('Esto es un ejemplo de frase para tokenizar.')}")
print(f"Token especial para el inicio de secuencia (CLS): {tokenizer.cls_token} (ID: {tokenizer.cls_token_id})")
print(f"Token especial para el final de secuencia (SEP): {tokenizer.sep_token} (ID: {tokenizer.sep_token_id})")
print(f"Token especial para padding (PAD): {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Tokenizador 'distilbert-base-uncased' cargado exitosamente.
Ejemplo de tokenización de una palabra: ['ho', '##la']
Ejemplo de tokenización de una frase: ['est', '##o', 'es', 'un', 'e', '##je', '##mp', '##lo', 'de', 'fra', '##se', 'para', 'token', '##iza', '##r', '.']
Token especial para el inicio de secuencia (CLS): [CLS] (ID: 101)
Token especial para el final de secuencia (SEP): [SEP] (ID: 102)
Token especial para padding (PAD): [PAD] (ID: 0)


In [ ]:
MAX_LENGTH = 54

def prepare_features(examples):
    # Tokenizar el texto completo del tweet
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_offsets_mapping=True # Necesario para alinear con selected_text
    )

    sentiment_to_label = {"negative": 0, "neutral": 1, "positive": 2}
    labels = sentiment_to_label[examples["sentiment"]]

    start_pos_val = 0
    end_pos_val = 0

    text = examples["text"]
    selected_text = examples["selected_text"]

    if pd.isna(selected_text):
       pass
    else:

        start_char_idx = text.find(selected_text)
        if start_char_idx != -1:
            end_char_idx = start_char_idx + len(selected_text)

            offset_mapping = tokenized_inputs["offset_mapping"]

            found_start_token_idx = -1
            found_end_token_idx = -1

            for i, (start_offset, end_offset) in enumerate(offset_mapping):
                if found_start_token_idx == -1 and start_offset <= start_char_idx < end_offset:
                    found_start_token_idx = i
                if start_offset < end_char_idx <= end_offset:
                    found_end_token_idx = i

                if found_start_token_idx != -1 and found_end_token_idx != -1:
                    break

            if found_start_token_idx != -1 and found_end_token_idx != -1:
                start_pos_val = found_start_token_idx
                end_pos_val = found_end_token_idx

    tokenized_inputs["labels"] = labels
    tokenized_inputs["start_positions"] = start_pos_val
    tokenized_inputs["end_positions"] = end_pos_val

    del tokenized_inputs["offset_mapping"]

    return tokenized_inputs

In [ ]:
processed_data = []
for index, row in df.iterrows():
    try:
        features = prepare_features(row)
        processed_data.append(features)
    except Exception as e:
        print(f"Error procesando fila {index}: {e}. Fila: {row['text']}")

print(f"\nSe han preprocesado {len(processed_data)} ejemplos.")


Se han preprocesado 27385 ejemplos.


In [ ]:
input_ids = torch.tensor([item['input_ids'] for item in processed_data])
attention_mask = torch.tensor([item['attention_mask'] for item in processed_data])
labels = torch.tensor([item['labels'] for item in processed_data])

start_positions = torch.tensor([item['start_positions'] for item in processed_data], dtype=torch.long)
end_positions = torch.tensor([item['end_positions'] for item in processed_data], dtype=torch.long)
print("\nFormas de los tensores de entrada/salida para BERT:")
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Mask shape: {attention_mask.shape}")
print(f"Labels (sentiment) shape: {labels.shape}")
print(f"Start Positions shape: {start_positions.shape}")
print(f"End Positions shape: {end_positions.shape}")


Formas de los tensores de entrada/salida para BERT:
Input IDs shape: torch.Size([27385, 54])
Attention Mask shape: torch.Size([27385, 54])
Labels (sentiment) shape: torch.Size([27385])
Start Positions shape: torch.Size([27385])
End Positions shape: torch.Size([27385])


In [ ]:
# Para verificar un ejemplo preprocesado (el primero)
print(f"\nPrimer Input IDs (ejemplo 1): {input_ids[0].tolist()}")
print(f"Primer Attention Mask (ejemplo 1): {attention_mask[0].tolist()}")
print(f"Primer Sentimiento (ejemplo 1): {labels[0].item()}")
print(f"Primer Start Positions (ejemplo 1): {start_positions[0].tolist()}")
print(f"Primer End Positions (ejemplo 1): {end_positions[0].tolist()}")


Primer Input IDs (ejemplo 1): [101, 8909, 2031, 5838, 2065, 1045, 2020, 2183, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Primer Attention Mask (ejemplo 1): [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Primer Sentimiento (ejemplo 1): 1
Primer Start Positions (ejemplo 1): 1
Primer End Positions (ejemplo 1): 7


In [ ]:
#paso 5 Procesamiento de data limpio para entrenamiento y validación
try:
    #Cargar y limpiar datos
    df_full = pd.read_csv('/content/drive/MyDrive/Aplicaciones de data science/dataminiconda/train_data_limpioof.csv')
    df_full.dropna(subset=['selected_text'], inplace=True)
    print(f"Dataset completo cargado y limpiado. Filas: {len(df_full)}")

    #Divide los datos
    train_df, val_df = train_test_split(df_full, test_size=0.2, random_state=42, stratify=df_full['sentiment'])
    print(f"Dividido en entrenamiento: {len(train_df)} filas")
    print(f"Dividido en validación: {len(val_df)} filas")

    #Función prepare_feature
    def prepare_features(examples):
        tokenized_inputs = tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=MAX_LENGTH,
            return_offsets_mapping=True
        )

        sentiment_to_label = {"negative": 0, "neutral": 1, "positive": 2}

        labels = [sentiment_to_label[s] for s in examples["sentiment"]]


        start_positions_batch = []
        end_positions_batch = []

        for i in range(len(examples["text"])):
            text = examples["text"][i]
            selected_text = examples["selected_text"][i]
            offset_mapping = tokenized_inputs["offset_mapping"][i]

            start_pos_val = 0
            end_pos_val = 0

            if pd.isna(selected_text):
                pass
            else:
                start_char_idx = text.find(selected_text)
                if start_char_idx != -1:
                    end_char_idx = start_char_idx + len(selected_text)

                    found_start_token_idx = -1
                    found_end_token_idx = -1

                    for j, (start_offset, end_offset) in enumerate(offset_mapping):
                        if found_start_token_idx == -1 and start_offset <= start_char_idx < end_offset:
                            found_start_token_idx = j
                        if start_offset < end_char_idx <= end_offset:
                            found_end_token_idx = j

                        if found_start_token_idx != -1 and found_end_token_idx != -1:
                            break

                    if found_start_token_idx != -1 and found_end_token_idx != -1:
                        start_pos_val = found_start_token_idx
                        end_pos_val = found_end_token_idx

            start_positions_batch.append(start_pos_val)
            end_positions_batch.append(end_pos_val)

        tokenized_inputs["labels"] = labels
        tokenized_inputs["start_positions"] = start_positions_batch
        tokenized_inputs["end_positions"] = end_positions_batch

        del tokenized_inputs["offset_mapping"]
        return tokenized_inputs

    #Procesamiento con datasets.Dataset
    # Especifica explícitamente las columnas a eliminar para evitar errores de "truth value"
    columns_to_remove = ["text", "selected_text", "sentiment", "textID"]

    train_hf_dataset = Dataset.from_pandas(train_df).map(
        prepare_features,
        batched=True,
        batch_size=16,
        remove_columns=columns_to_remove,
        load_from_cache_file=False
    )

    val_hf_dataset = Dataset.from_pandas(val_df).map(
        prepare_features,
        batched=True,
        batch_size=16,
        remove_columns=columns_to_remove,
        load_from_cache_file=False
    )

    #Convertir a tensores
    def dataset_to_tensors(dataset):
        return TensorDataset(
            torch.tensor(dataset['input_ids'], dtype=torch.long),
            torch.tensor(dataset['attention_mask'], dtype=torch.long),
            torch.tensor(dataset['labels'], dtype=torch.long),
            torch.tensor(dataset['start_positions'], dtype=torch.long),
            torch.tensor(dataset['end_positions'], dtype=torch.long)
        )

    train_dataset = dataset_to_tensors(train_hf_dataset)
    val_dataset = dataset_to_tensors(val_hf_dataset)

    #Crear DataLoaders
    BATCH_SIZE = 16

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        pin_memory=False
    )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )

    print(f"\nNúmero de ejemplos de entrenamiento: {len(train_dataset)}")
    print(f"Número de ejemplos de validación: {len(val_dataset)}")

except Exception as e:
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

Dataset completo cargado y limpiado. Filas: 27385
Dividido en entrenamiento: 21908 filas
Dividido en validación: 5477 filas


Map:   0%|          | 0/21908 [00:00<?, ? examples/s]

Map:   0%|          | 0/5477 [00:00<?, ? examples/s]


Número de ejemplos de entrenamiento: 21908
Número de ejemplos de validación: 5477


In [ ]:
#Usamos DistilBert
MODEL_NAME = 'distilbert-base-uncased'

#Cargamos tokenizador específico para DistilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

#clase del modelo para DistilBERT
class DistilBertForSentimentAndExtraction(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.classifier = nn.Linear(config.dim, config.num_labels)
        self.qa_outputs = nn.Linear(config.dim, 2)

    def forward(self, input_ids, attention_mask, labels=None, start_positions=None, end_positions=None):
        outputs = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )

        sequence_output = outputs.last_hidden_state

        # Para el sentimiento, usamos el token [CLS]
        pooled_output = sequence_output[:, 0]
        sentiment_logits = self.classifier(pooled_output)

        # Para el span extraction
        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        #Calculamos la pérdida
        loss = None
        if labels is not None and start_positions is not None and end_positions is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss_sentiment = loss_fct(sentiment_logits.view(-1, self.num_labels), labels.view(-1))
            loss_start = loss_fct(start_logits, start_positions)
            loss_end = loss_fct(end_logits, end_positions)
            loss = loss_sentiment + (loss_start + loss_end)/2

        elif labels is not None:
             loss_fct = nn.CrossEntropyLoss()
             loss = loss_fct(sentiment_logits.view(-1, self.num_labels), labels.view(-1))

        return {
            "loss": loss,
            "sentiment_logits": sentiment_logits,
            "start_logits": start_logits,
            "end_logits": end_logits
        }


In [ ]:
#Carga de configuración y modelo
config = DistilBertConfig.from_pretrained(MODEL_NAME)
config.num_labels = 3
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")
model = DistilBertForSentimentAndExtraction(config)
model.to(device)
print("Model loaded and moved to device.")

Using device: cuda
Model loaded and moved to device.


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
EPOCHS = 3

total_steps = len(train_dataloader) * EPOCHS

# Crea un programador de tasa de aprendizaje con calentamiento lineal
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"Optimizador AdamW configurado con LR: 2e-5, Épocas: {EPOCHS}.")
print(f"Número total de pasos de entrenamiento: {total_steps}")

Optimizador AdamW configurado con LR: 2e-5, Épocas: 3.
Número total de pasos de entrenamiento: 4110


In [ ]:
# Antes de entrenar
gc.collect()
torch.cuda.empty_cache()
scaler = GradScaler()

/tmp/ipython-input-17-4024808779.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
# Función para calcular la precisión (accuracy) para la clasificación de sentimiento
def calculate_sentiment_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

seed_val = 42
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
np.random.seed(seed_val)
random.seed(seed_val)

# Bucle de entrenamiento
scaler = GradScaler()
accumulation_steps = 2

for epoch_i in range(0, EPOCHS):
    print(f'\n======== Época {epoch_i + 1} / {EPOCHS} ========')
    print('Entrenando...')

    total_train_loss = 0
    model.train()
    optimizer.zero_grad()

    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_start_positions = batch[3].to(device)
        b_end_positions = batch[4].to(device)

        with autocast():
            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_attention_mask,
                labels=b_labels,
                start_positions=b_start_positions,
                end_positions=b_end_positions
            )
            loss = outputs["loss"] / accumulation_steps

        scaler.scale(loss).backward()
        total_train_loss += loss.item() * accumulation_steps

        if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_dataloader):

            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

    # Calcular la pérdida promedio
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"\n  Pérdida de entrenamiento promedio: {avg_train_loss:.2f}")

    # --- Evaluación ---
    print("\nCorriendo evaluación...")
    model.eval()

    total_eval_loss = 0
    total_eval_sentiment_accuracy = 0
    all_preds = []
    all_labels = []

    for batch in tqdm(val_dataloader, desc="Evaluating"):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_start_positions = batch[3].to(device)
        b_end_positions = batch[4].to(device)

        with torch.no_grad():
            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_attention_mask,
                labels=b_labels,
                start_positions=b_start_positions,
                end_positions=b_end_positions
            )

        loss = outputs["loss"]
        sentiment_logits = outputs["sentiment_logits"]
        # Obtener predicciones
        preds = torch.argmax(sentiment_logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(b_labels.cpu().numpy())

        total_eval_loss += loss.item()
        sentiment_logits = sentiment_logits.detach().cpu().numpy()
        b_labels = b_labels.to('cpu').numpy()
        total_eval_sentiment_accuracy += calculate_sentiment_accuracy(sentiment_logits, b_labels)

    # Reportar métricas
    avg_eval_loss = total_eval_loss / len(val_dataloader)
    avg_eval_sentiment_accuracy = total_eval_sentiment_accuracy / len(val_dataloader)
    print(f"  Pérdida de validación: {avg_eval_loss:.2f}")
    print(f"  Precisión de sentimiento: {avg_eval_sentiment_accuracy:.2f}")

print("\n¡Entrenamiento completado!")


======== Época 1 / 3 ========
Entrenando...


/tmp/ipython-input-18-4000195558.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Training:   0%|          | 0/1370 [00:00<?, ?it/s]

/tmp/ipython-input-18-4000195558.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



  Pérdida de entrenamiento promedio: 2.86

Corriendo evaluación...


...

  Pérdida de validación: 2.13
  Precisión de sentimiento: 0.70

======== Época 3 / 3 ========
Entrenando...


Training:   0%|          | 0/1370 [00:00<?, ?it/s]


  Pérdida de entrenamiento promedio: 1.92

Corriendo evaluación...


Evaluating:   0%|          | 0/343 [00:00<?, ?it/s]

  Pérdida de validación: 2.11
  Precisión de sentimiento: 0.71

¡Entrenamiento completado!


## Save the Trained Model

Let's save the state dictionary of your trained `DistilBertForSentimentAndExtraction` model. This allows you to load the trained weights later without needing to retrain the model.

In [ ]:
output_dir = '/content/drive/MyDrive/Aplicaciones de data science/dataminiconda/trained_model' # Choose a path in your Google Drive
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_save_path = os.path.join(output_dir, 'sentiment_extraction_model.pth')

# Save the model's
if 'model' in locals():
    try:
        torch.save(model.state_dict(), model_save_path)
        print(f"Model state dictionary saved to {model_save_path}")
    except Exception as e:
        print(f"An error occurred while saving the model: {e}")
else:
    print("Model not found. Please ensure the model has been trained.")

Model state dictionary saved to /content/drive/MyDrive/Aplicaciones de data science/dataminiconda/trained_model/sentiment_extraction_model.pth


## Load the Trained Model for Inference

To use the trained model with the `transformers.pipeline`, we need to load a standard Hugging Face model and then load the state dictionary from our saved model.

In [ ]:
from transformers import DistilBertForSequenceClassification
model_save_path = '/content/drive/MyDrive/Aplicaciones de data science/dataminiconda/trained_model/sentiment_extraction_model.pth'

MODEL_NAME = 'distilbert-base-uncased'
try:
    loaded_model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3) # Assuming 3 sentiment labels
    state_dict = torch.load(model_save_path, map_location=device)
    sentiment_state_dict = {k: v for k, v in state_dict.items() if 'classifier' in k or 'distilbert' in k}
    loaded_model.load_state_dict(sentiment_state_dict, strict=False) # Use strict=False to ignore non-matching keys (like qa_outputs)
    loaded_model.to(device)
    loaded_model.eval()

    print("Trained model loaded successfully.")

except FileNotFoundError:
    print(f"Error: Model state dictionary not found at {model_save_path}. Please check the path.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trained model loaded successfully.


PRIMERA PRUEBA : AANALIZADOR DE TWIITS , COMENTARIOS O QUEJAS PARA EL RENDIMIENTO DEL PRODUCTO O NEGOCIO. FORMA MANUAL

In [ ]:
from transformers import pipeline

def analizar_tweet():
    texto = input("Ingresa un tweet para analizar: ")

    if 'model' not in globals() or 'tokenizer' not in globals() or 'MAX_LENGTH' not in globals() or 'device' not in globals():
        print("Error: Model, tokenizer, MAX_LENGTH, or device not initialized.")
        return

    model.eval() # Set the model to evaluation mode

    # Tokenize the input tweet
    inputs = tokenizer(
        texto,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_offsets_mapping=True # Needed to map token indices back to characters
    ).to(device) # Move inputs to the same device as the model

    offset_mapping = inputs.pop("offset_mapping").squeeze(0) # Remove batch dimension


    with torch.no_grad():
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
            # No labels, start_positions, end_positions needed for inference
        )

    # Get sentiment prediction
    sentiment_logits = outputs["sentiment_logits"]
    predicted_label_id = torch.argmax(sentiment_logits, dim=1).item()

    # Map label ID back to sentiment string
    id_to_sentiment = {0: "negative", 1: "neutral", 2: "positive"}
    predicted_sentiment = id_to_sentiment[predicted_label_id]
     # Get confidence score
    confidence = torch.softmax(sentiment_logits, dim=1)[0][predicted_label_id].item()


    # Get span extraction prediction
    start_logits = outputs["start_logits"].squeeze(0) # Remove batch dimension
    end_logits = outputs["end_logits"].squeeze(0)   # Remove batch dimension

    # Find the token indices with the highest start and end logits
    predicted_start_token_idx = torch.argmax(start_logits).item()
    predicted_end_token_idx = torch.argmax(end_logits).item()

    if predicted_start_token_idx > predicted_end_token_idx:
         predicted_start_token_idx, predicted_end_token_idx = predicted_end_token_idx, predicted_start_token_idx

    predicted_selected_text = ""
    # Check if the predicted indices are within the valid range of offset_mapping
    if predicted_start_token_idx >= 0 and predicted_end_token_idx < len(offset_mapping) and predicted_start_token_idx <= predicted_end_token_idx:
         start_char = offset_mapping[predicted_start_token_idx][0].item()
         end_char = offset_mapping[predicted_end_token_idx][1].item()

         if start_char >= 0 and end_char <= len(texto) and start_char <= end_char:
            predicted_selected_text = texto[start_char:end_char]
         else:
             # If offsets are invalid, maybe the prediction was for special tokens, default to empty or full text
             predicted_selected_text = texto
    else:

        predicted_selected_text = texto # Or ""


    print(f"\nSentimiento: {predicted_sentiment} (Confianza: {confidence:.2%})")
    print(f"Frase clave: {predicted_selected_text}")

# Ejecutar en bucle
while True:
    analizar_tweet()

KeyboardInterrupt: Interrupted by user

SEGUNDA PRUEBA : CREAR UN DASHBOARD PARA UNA SIMULADOR DE CRISIS DE UN PRODUCTO EN UN NEGOCIO (EN PROCESO)

In [ ]:
!pip install streamlit pyngrok

from pyngrok import ngrok

!ngrok authtoken 2zYvhAtUy56PyJYWSBetCnWpz8W_7qCbmGQCfqoDFJ8oaviRk


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile crisis_simulator.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from transformers import BertTokenizerFast, DistilBertConfig

# Clase del modelo (copiada del entrenamiento)
class DistilBertForSentimentAndExtraction(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_labels = config.num_labels
        from transformers import DistilBertModel
        self.distilbert = DistilBertModel(config)
        self.classifier = nn.Linear(config.dim, config.num_labels)
        self.qa_outputs = nn.Linear(config.dim, 2)

    def forward(self, input_ids, attention_mask, labels=None, start_positions=None, end_positions=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        sequence_output = outputs.last_hidden_state
        pooled_output = sequence_output[:, 0]
        sentiment_logits = self.classifier(pooled_output)
        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        loss = None
        if labels is not None and start_positions is not None and end_positions is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss_sentiment = loss_fct(sentiment_logits.view(-1, self.num_labels), labels.view(-1))
            loss_start = loss_fct(start_logits, start_positions)
            loss_end = loss_fct(end_logits, end_positions)
            loss = loss_sentiment + (loss_start + loss_end) / 2
        elif labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(sentiment_logits.view(-1, self.num_labels), labels.view(-1))
        return {
            "loss": loss,
            "sentiment_logits": sentiment_logits,
            "start_logits": start_logits,
            "end_logits": end_logits
        }

# Cargar modelo
@st.cache_resource
def load_model():
    tokenizer = BertTokenizerFast.from_pretrained('distilbert-base-uncased')
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    config.num_labels = 3
    model = DistilBertForSentimentAndExtraction(config)
    model.load_state_dict(torch.load('/content/drive/MyDrive/Aplicaciones de data science/dataminiconda/trained_model/sentiment_extraction_model.pth', map_location='cpu'))
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

# Predicción
def predict_crisis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=64, return_token_type_ids=False)
    with torch.no_grad():
        outputs = model(**inputs)
    sentiment_probs = torch.softmax(outputs['sentiment_logits'], dim=1)
    sentiment = ["NEGATIVO", "NEUTRAL", "POSITIVO"][torch.argmax(sentiment_probs)]
    start_idx = torch.argmax(outputs['start_logits'])
    end_idx = torch.argmax(outputs['end_logits'])
    key_phrase = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx+1])
    return {
        "sentiment": sentiment,
        "confidence": torch.max(sentiment_probs).item(),
        "key_phrase": key_phrase
    }

# Dataset de ejemplo
def generate_sample_data():
    dates = pd.date_range(start="2023-01-01", periods=24, freq='H')
    data = {
        "hora": dates,
        "tweets_negativos": np.random.randint(1, 10, 24),
        "tweets_positivos": np.random.randint(5, 15, 24),
        "palabras_clave": ["servicio", None, "app", None, "pago", "demora"] * 4
    }
    return pd.DataFrame(data)

# Streamlit UI
st.set_page_config(layout="wide")
st.title("Simulador de Crisis Temprana")

tab1, tab2 = st.tabs(["Análisis en Tiempo Real", "Dashboard Histórico"])

with tab1:
    st.subheader("Monitor Live")
    user_input = st.text_area("Ingresa un tweet para analizar:", "El servicio es terrible y nunca responden a mis reclamos...")
    if st.button("Analizar"):
        result = predict_crisis(user_input)
        cols = st.columns(3)
        cols[0].metric("Sentimiento", result['sentiment'])
        cols[1].metric("Confianza", f"{result['confidence']:.2%}")
        cols[2].metric("Frase Clave", result['key_phrase'])
        if result['sentiment'] == "NEGATIVO" and result['confidence'] > 0.7:
            st.error("ALERTA: Posible crisis detectada!")
            st.progress(result['confidence'])

with tab2:
    st.subheader("Tendencias de Crisis")
    df = generate_sample_data()
    fig, ax = plt.subplots()
    df.plot(x='hora', y=['tweets_negativos', 'tweets_positivos'], kind='line', ax=ax)
    st.pyplot(fig)
    crisis_words = df['palabras_clave'].value_counts().dropna()
    st.bar_chart(crisis_words)


Overwriting crisis_simulator.py


In [ ]:
from pyngrok import ngrok

# Cerrar todos los túneles abiertos
ngrok.kill()

In [ ]:
from pyngrok import ngrok

# Abrir túnel al puerto donde corre Streamlit
public_url = ngrok.connect(8501)
print(f"URL pública de tu app: {public_url}")

# Ejecutar Streamlit en segundo plano
!streamlit run crisis_simulator.py &> /dev/null &


URL pública de tu app: NgrokTunnel: "https://8c1128b59974.ngrok-free.app" -> "http://localhost:8501"
